## import pkg

In [1]:
# import sktime

### for preprocessing, we have different methods for different column (horizontal)

In [5]:
from sklearn.compose import ColumnTransformer

### for model training, we have step-by-step methods (vertical)

In [6]:
from sklearn.pipeline import make_pipeline

### for Nan Numerical data in dp, i use KNNImputer

In [7]:
from sklearn.impute import KNNImputer

### for Nan categorical data, i use sklearn.impute.SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer

### for categorical data, i use pd.series.astype("category")

df = pd.DataFrame({"A": list("abca"), "B": list("bccd")})
convert just columns "a" and "b"
df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)

### for numerical data, i use stdscaler

In [5]:
from sklearn.preprocessing import StandardScaler

## load dataset

In [1]:
path = "D:/202209/tbrain/訓練資料集_first/"
import os
os.chdir(path)
# import glob
# print(glob.glob(os.path.join(path, "*.csv")))
## 備註
## 這邊利用 os.path.join 來連接指定字串形成路徑，因為不同系統下的分隔符號可能不同

In [2]:
import pandas as pd

In [3]:
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv")
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv")
custinfo = pd.read_csv("public_train_x_custinfo_full_hashed.csv")
dp = pd.read_csv("public_train_x_dp_full_hashed.csv")
remit = pd.read_csv("public_train_x_remit1_full_hashed.csv")
alert_public = pd.read_csv("public_x_alert_date.csv")
alert_train = pd.read_csv("train_x_alert_date.csv")
y = pd.read_csv("train_y_answer.csv")

## Prepare training/testing target

In [11]:
train = y.join(alert_train.set_index('alert_key'), on='alert_key')
train = train.join(custinfo.set_index('alert_key'), on='alert_key')
train.head()

,alert_key,sar_flag,date,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3


In [12]:
test = y.join(alert_public.set_index('alert_key'), on='alert_key')
test = test.join(custinfo.set_index('alert_key'), on='alert_key')
test.head()

,alert_key,sar_flag,date,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,NaN,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,NaN,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,NaN,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,NaN,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,NaN,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3


In [ ]:
alert_join = pd.concat([alert_train, alert_public],axis=0)
alert_join = alert_join.join(custinfo.set_index('alert_key'), on='alert_key')
alert_join.iloc[23900:23910]

In [ ]:
alert_join_y = alert_join.join(y.set_index('alert_key'), on='alert_key')
alert_join_y1 = alert_join_y[alert_join_y['sar_flag']==1.0]
alert_join_y1.nunique()

## prepare sub-dataset

In [13]:
import numpy as np

In [15]:
imp_occu = SimpleImputer(strategy='constant', fill_value=40)
custinfo[['occupation_code']] = imp_occu.fit_transform(custinfo[['occupation_code']])

In [16]:
imp_fisc = SimpleImputer(strategy='constant', fill_value=30)
dp[['fiscTxId']] = imp_occu.fit_transform(dp[['fiscTxId']])

In [17]:
imp_branch = SimpleImputer(strategy='constant', fill_value=400)
dp[['txbranch']] = imp_occu.fit_transform(dp[['txbranch']])

### transfer the categorical datas into pd.Categorical

In [18]:
ccba[['byymm']] = pd.Categorical(ccba[['byymm']], ordered=True)

cdtx[['date']] = pd.Categorical(cdtx[['date']], ordered=True)
cdtx[['country', 'cur_type']] = pd.Categorical(cdtx[['country', 'cur_type']])

custinfo[['risk_rank', 'AGE']] = pd.Categorical(custinfo[['risk_rank', 'AGE']], ordered=True)
custinfo[['occupation_code']] = pd.Categorical(custinfo[['occupation_code']])

dp[['tx_date', 'tx_time']] = pd.Categorical(dp[['tx_date', 'tx_time']], ordered=True)
dp[['debit_credit', 'tx_type', 'info_asset_code', 'fiscTxId', 'txbranch', 'cross_bank', 'ATM']] = pd.Categorical(dp[['debit_credit', 'tx_type', 'info_asset_code', 'fiscTxId', 'txbranch', 'cross_bank', 'ATM']])

remit[['trans_date']] = pd.Categorical(remit[['trans_date']], ordered=True)

## prepare dp

In [19]:
# calculate into NTD
dp['NTD'] = dp['tx_amt']*dp['exchg_rate'].round(2)

# is it originally used NTD?
dp['isNTD'] = dp.loc[dp["exchg_rate"] == 1.0, "isNTD"] = 1
dp['isNTD'] = dp.loc[dp["exchg_rate"] != 1.0, "isNTD"] = 0

dp[dp['exchg_rate']!=1.0].head()

,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,NTD,isNTD
199,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,debit_credit,tx_date,tx_time,tx_type,3715.0,27.83,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,103388.45,0
200,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,debit_credit,tx_date,tx_time,tx_type,5029.0,27.80,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,139806.20,0
201,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,debit_credit,tx_date,tx_time,tx_type,3715.0,27.84,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,103425.60,0
202,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,debit_credit,tx_date,tx_time,tx_type,8428.0,27.84,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,234635.52,0
205,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,debit_credit,tx_date,tx_time,tx_type,2064.0,27.63,info_asset_code,fiscTxId,txbranch,cross_bank,ATM,57028.32,0


In [4]:
# dp[dp['tx_amt'].isnull()][:10]

In [ ]:
dp[dp['tx_amt'].isnull()]

In [ ]:
imputer_branch = SimpleImputer(strategy='median')

dp["tx_amt"] = imputer_branch.fit_transform(dp[["tx_amt"]])


In [25]:
# booooom
# from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2, weights="uniform")

# dp["tx_amt"] = imputer.fit_transform(dp[["tx_amt"]]).ravel()
